In [1]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns
from tqdm.notebook import tqdm, trange
import time
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from datetime import timedelta

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,\
AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,\
ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
#
import xgboost as xgb

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CVolpacchio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CVolpacchio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
view_df = pd.read_csv('c:/Users/CVolpacchio/Documents/revenuebyrs_view_2021.csv')
line_df = pd.read_csv('c:/Users/CVolpacchio/Documents/db04_line_2021.csv')
print(view_df.columns,line_df.columns)

C:\Users\CVolpacchio\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (28,51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\CVolpacchio\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['Date', 'WeekofYear', 'PeriodNumber', 'PeriodDay', 'Quarter', 'Year',
       'Amount', 'NumInvoices', 'RevenueType_ID', 'RevenueType', 'StaffID',
       'FullName', 'LastName', 'FirstName', 'MI', 'LocationNumber', 'AcqDate',
       'Location', 'RegionalManager', 'Region', 'LocationType', 'Department',
       'Number', 'Installation_GUID', 'SVPFullName', 'Email', 'SVPCellPhone',
       'SVPOfficePhone', 'Vendor_ID', 'Commissionable_IND', 'Location_ID',
       'Location.1', 'ShortName', 'Code', 'AcquisitionDate', 'Manager_AdminID',
       'AcqOrder', 'Type', 'Metro_ID', 'Valid', 'LinkedServer', 'DBType',
       'DBName', 'GL_Code', 'LocationGroup_ID', 'LocationNumber.1',
       'Hospital_Name_TXT', 'Time_Zone_TXT', 'Purchase_Rollup_Group_ID',
       'isMultiCenter', 'Location_ID_Parent', 'PMS_Site_ID',
       'PMS_Practice_ID'],
      dtype='object') Index(['Location_ID', 'Site_id', 'TransNumber', 'InvoiceNumber', 'InvoiceDate',
       'LineItemDate', 'ProviderID', 'ClientID', 'Pa

In [3]:
view_df.head()

,Date,WeekofYear,PeriodNumber,PeriodDay,Quarter,Year,Amount,NumInvoices,RevenueType_ID,RevenueType,...,GL_Code,LocationGroup_ID,LocationNumber.1,Hospital_Name_TXT,Time_Zone_TXT,Purchase_Rollup_Group_ID,isMultiCenter,Location_ID_Parent,PMS_Site_ID,PMS_Practice_ID
0,2021-01-15,2,1,12,1,2021,15.67,1.00,1,PMS,...,"010,015",1,1,CVS - Charlotte,EST,1,0,1,0,CVS
1,2021-01-21,3,1,18,1,2021,45.15,1.00,1,PMS,...,"010,015",1,1,CVS - Charlotte,EST,1,0,1,0,CVS
2,2021-01-14,2,1,11,1,2021,551.58,1.00,1,PMS,...,"010,015",1,1,CVS - Charlotte,EST,1,0,1,0,CVS
3,2021-01-20,3,1,17,1,2021,777.19,0.23,1,PMS,...,"010,015",1,1,CVS - Charlotte,EST,1,0,1,0,CVS
4,2021-01-16,2,1,13,1,2021,1638.63,8.10,1,PMS,...,"010,015",1,1,CVS - Charlotte,EST,1,0,1,0,CVS


In [4]:
# looking for missing days
view_df['Date'].value_counts().index
line_df['InvoiceDate'] = pd.to_datetime(line_df['InvoiceDate'])
line_df['InvoiceDate'].value_counts().index
line_df['InvoiceDate'].dt.date.value_counts().index

print(len(view_df['Date'].value_counts().index),len(line_df['InvoiceDate'].dt.date.value_counts().index))
#[x for x in ]

136 42


In [5]:
view_df['Date'].describe()

count         286620
unique           136
top       2021-04-30
freq            2604
Name: Date, dtype: object

In [6]:
#line_df['InvoiceDate'].dt.date.values
line_df['InvoiceDate'].dt.date.value_counts().index

Index([2021-01-18, 2021-01-04, 2021-01-26, 2021-01-25, 2021-01-21, 2021-02-10,
       2021-01-13, 2021-01-05, 2021-02-05, 2021-02-08, 2021-01-29, 2021-01-15,
       2021-01-11, 2021-01-22, 2021-01-14, 2021-02-09, 2021-01-06, 2021-01-12,
       2021-01-20, 2021-01-19, 2021-01-07, 2021-02-03, 2021-01-08, 2021-02-04,
       2021-02-01, 2021-01-27, 2021-01-28, 2021-02-02, 2021-02-11, 2021-01-31,
       2021-02-06, 2021-01-30, 2021-01-16, 2021-01-23, 2021-01-09, 2021-01-02,
       2021-02-07, 2021-01-03, 2021-01-24, 2021-01-10, 2021-01-17, 2021-01-01],
      dtype='object')

In [7]:
view_df['Date'] = pd.to_datetime(view_df['Date'])
view_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286620 entries, 0 to 286619
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Date                      286620 non-null  datetime64[ns]
 1   WeekofYear                286620 non-null  int64         
 2   PeriodNumber              286620 non-null  int64         
 3   PeriodDay                 286620 non-null  int64         
 4   Quarter                   286620 non-null  int64         
 5   Year                      286620 non-null  int64         
 6   Amount                    286620 non-null  float64       
 7   NumInvoices               286620 non-null  float64       
 8   RevenueType_ID            286620 non-null  int64         
 9   RevenueType               286620 non-null  object        
 10  StaffID                   286616 non-null  object        
 11  FullName                  286620 non-null  object        
 12  La

In [8]:
view_dates = pd.to_datetime(list(view_df['Date'].value_counts().index))
line_dates = pd.to_datetime(list(line_df['InvoiceDate'].dt.date.value_counts().index))
print(len([x for x in view_dates if x not in line_dates]))
[x for x in view_dates if x not in line_dates]

97


[Timestamp('2021-04-30 00:00:00'),
 Timestamp('2021-05-11 00:00:00'),
 Timestamp('2021-04-12 00:00:00'),
 Timestamp('2021-04-19 00:00:00'),
 Timestamp('2021-05-17 00:00:00'),
 Timestamp('2021-03-22 00:00:00'),
 Timestamp('2021-04-26 00:00:00'),
 Timestamp('2021-03-31 00:00:00'),
 Timestamp('2021-03-01 00:00:00'),
 Timestamp('2021-05-10 00:00:00'),
 Timestamp('2021-03-29 00:00:00'),
 Timestamp('2021-03-23 00:00:00'),
 Timestamp('2021-05-05 00:00:00'),
 Timestamp('2021-05-03 00:00:00'),
 Timestamp('2021-03-19 00:00:00'),
 Timestamp('2021-05-12 00:00:00'),
 Timestamp('2021-03-08 00:00:00'),
 Timestamp('2021-03-16 00:00:00'),
 Timestamp('2021-05-14 00:00:00'),
 Timestamp('2021-04-21 00:00:00'),
 Timestamp('2021-04-15 00:00:00'),
 Timestamp('2021-04-01 00:00:00'),
 Timestamp('2021-05-13 00:00:00'),
 Timestamp('2021-04-28 00:00:00'),
 Timestamp('2021-05-06 00:00:00'),
 Timestamp('2021-04-29 00:00:00'),
 Timestamp('2021-04-20 00:00:00'),
 Timestamp('2021-05-04 00:00:00'),
 Timestamp('2021-03-

In [9]:
# Line : Gross amount >== net??; since not ALL locs are present will join on nmbr
# View : 'NumInvoices' not integer?
### Possibly better to grab view by invoice aggregation?
view_group = view_df.groupby(['Date',
                              'Location_ID']).agg({'Amount':sum,
                                         'NumInvoices':sum})
view_group.reset_index(inplace=True)
line_group = line_df.groupby(['InvoiceDate',
                              'Location_ID']).agg({'NetAmount':sum,
                                                'GrossAmount':sum,
                                                'InvoiceNumber':'nunique'})
line_group.reset_index(inplace=True)
master = view_group.merge(line_group, how = 'inner',
                left_on = ['Date','Location_ID'],
                right_on = ['InvoiceDate','Location_ID'])
master.head()

,Date,Location_ID,Amount,NumInvoices,InvoiceDate,NetAmount,GrossAmount,InvoiceNumber
0,2021-01-04,1,19711.29,41.07,2021-01-04,19723.52,19735.26,42
1,2021-01-04,3,33547.14,35.94,2021-01-04,33890.61,33890.61,38
2,2021-01-04,5,32115.63,61.96,2021-01-04,31975.83,31985.16,63
3,2021-01-04,20,14736.75,29.01,2021-01-04,14736.75,14736.75,1
4,2021-01-04,28,6709.77,34.00,2021-01-04,6709.77,0.00,34


In [10]:
master['NumInvoices'] = [round(x,0) for x in master['NumInvoices']]
master['Amount'] = [round(x,0) for x in master['Amount']]
master['NetAmount'] = [round(x,0) for x in master['NetAmount']]

master['View_less_line_Invoice'] = master['NumInvoices'] - master['InvoiceNumber']
master['View_less_line_Rev_Diff'] = master['Amount'] - master['NetAmount']
master.to_csv('c:/Users/CVolpacchio/Documents/line_vs_view.csv')
master.head()

,Date,Location_ID,Amount,NumInvoices,InvoiceDate,NetAmount,GrossAmount,InvoiceNumber,View_less_line_Invoice,View_less_line_Rev_Diff
0,2021-01-04,1,19711.0,41.0,2021-01-04,19724.0,19735.26,42,-1.0,-13.0
1,2021-01-04,3,33547.0,36.0,2021-01-04,33891.0,33890.61,38,-2.0,-344.0
2,2021-01-04,5,32116.0,62.0,2021-01-04,31976.0,31985.16,63,-1.0,140.0
3,2021-01-04,20,14737.0,29.0,2021-01-04,14737.0,14736.75,1,28.0,0.0
4,2021-01-04,28,6710.0,34.0,2021-01-04,6710.0,0.00,34,0.0,0.0


In [11]:
# Line : Gross amount >== net??; since not ALL locs are present will join on nmbr
# View : 'NumInvoices' not integer?
### Possibly better to grab view by invoice aggregation?
docview_group = view_df.groupby(['Date',
                              'StaffID',
                              'Location_ID']).agg({'Amount':sum,
                                         'NumInvoices':sum})
docview_group.reset_index(inplace=True)
docline_group = line_df.groupby(['InvoiceDate',
                                 'ProviderID',
                              'Location_ID']).agg({'NetAmount':sum,
                                                'GrossAmount':sum,
                                                'InvoiceNumber':'nunique'})
docline_group.reset_index(inplace=True)
master = docview_group.merge(docline_group, how = 'inner',
                left_on = ['Date','StaffID','Location_ID'],
                right_on = ['InvoiceDate','ProviderID','Location_ID'])
master

,Date,StaffID,Location_ID,Amount,NumInvoices,InvoiceDate,ProviderID,NetAmount,GrossAmount,InvoiceNumber
0,2021-01-04,0,39,998.96,11.00,2021-01-04,0,998.96,1004.52,14
1,2021-01-04,0,202,336.16,8.00,2021-01-04,0,336.16,336.16,9
2,2021-01-04,1,47,1218.88,8.00,2021-01-04,1,1218.88,1218.88,8
3,2021-01-04,1,68,3021.30,29.00,2021-01-04,1,2857.82,2249.46,30
4,2021-01-04,1,71,1909.03,13.00,2021-01-04,1,510.15,837.84,12
...,...,...,...,...,...,...,...,...,...,...
25844,2021-02-11,WW,106,1354.92,4.00,2021-02-11,WW,1202.62,1202.62,6
25845,2021-02-11,WW,322,6482.48,15.98,2021-02-11,WW,6482.48,6482.48,21
25846,2021-02-11,WWRX,322,698.53,14.00,2021-02-11,WWRX,698.53,698.53,16
25847,2021-02-11,ZJM,41,2997.10,10.32,2021-02-11,ZJM,2997.10,2997.10,2


In [12]:
master['NumInvoices'] = [round(x,0) for x in master['NumInvoices']]
master['Amount'] = [round(x,0) for x in master['Amount']]
master['NetAmount'] = [round(x,0) for x in master['NetAmount']]

master['View_less_line_Invoice'] = master['NumInvoices'] - master['InvoiceNumber']
master['View_less_line_Rev_Diff'] = master['Amount'] - master['NetAmount']
master.to_csv('c:/Users/CVolpacchio/Documents/doc_line_vs_view.csv')
master.head()

,Date,StaffID,Location_ID,Amount,NumInvoices,InvoiceDate,ProviderID,NetAmount,GrossAmount,InvoiceNumber,View_less_line_Invoice,View_less_line_Rev_Diff
0,2021-01-04,0,39,999.0,11.0,2021-01-04,0,999.0,1004.52,14,-3.0,0.0
1,2021-01-04,0,202,336.0,8.0,2021-01-04,0,336.0,336.16,9,-1.0,0.0
2,2021-01-04,1,47,1219.0,8.0,2021-01-04,1,1219.0,1218.88,8,0.0,0.0
3,2021-01-04,1,68,3021.0,29.0,2021-01-04,1,2858.0,2249.46,30,-1.0,163.0
4,2021-01-04,1,71,1909.0,13.0,2021-01-04,1,510.0,837.84,12,1.0,1399.0


In [137]:
# instances of invoice counts not aligning but revenue tying may be my count approach
# or the NumInvoices count approach
